In [1]:
#importing necessary packages
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM,Dropout
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
#function to generate the required dataset
def generate_samples(length=50,numberOfValues=100000):
    '''
    Generate the required dataset
    
    Args:
        length: Length of single binary string
        numberOfValues: by default is set to 100000, as per the problem statement
                        else can be se4t to any comfortable number depending on 
                        the developing state.
    Returns:
        Numpy array of binary strings and array of parity bit labels
    '''
    # Generate random strings of length 50
    if length == 50:
        data = np.random.randint(2, size=(numberOfValues, length))
        labels = [0 if sum(i) % 2 == 0 else 1 for i in data]
    # Generate random strings of variable length
    else:
        data = []
        labels = []
        for i in range(numberOfValues):
            # Choose random length
            length = np.random.randint(1, 51)
            data.append(np.random.randint(2, size=(length)))
            labels.append(0 if sum(data[i]) % 2 == 0 else 1)
        data = np.asarray(data)
        # Pad binary strings with 0's to make sequence length same for all
        data = pad_sequences(data, maxlen=50, padding='pre')

    labels = np.asarray(labels)
    train_size = data.shape[0]
    size = int(train_size * 0.20)
    # Split data into train/test sets
    X_test = data[:size]
    X_train = data[size:]
    y_test = labels[:size]
    y_train = labels[size:]
    # Expand dimension to feed into LSTM layer
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)

    return X_train, y_train, X_test, y_test

In [4]:
def build_model():
    ''' Build LSTM model using Keras
    Args:
        None
    Returns:
        Compiled LSTM model
    '''
    model = Sequential()
    model.add(LSTM(32, input_shape=(50, 1)))
    model.add(Dense(1, activation='relu'))
    model.add(Dropout(0.2))
    # Display model summary
    model.summary()
    model.compile('adam', loss='binary_crossentropy', metrics=['acc'])

    return model

In [5]:
def plot_model(history):
    ''' Plot model accuracy and loss
    Args:
        history: Keras dictionary contatining training/validation loss/acc
    Returns:
        Plots model's training/validation loss and accuracy history
    '''
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(loss) + 1)

    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.figure()
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()
    return

In [ ]:
'''
To test the first waarmup 1 case :
fixed length of binary strings=50
'''
n_epochs=25
batch_size=32
validation_split=0.2
flag_shuffle=True

X_train, y_train, X_test, y_test=generate_samples()
XOR_model=build_model()

history=XOR_model.fit(X_train,y_train,epochs=n_epochs,
                      batch_size=batch_size,
                      validation_split=validation_split,
                      shuffle=flag_shuffle)
# Evaluate model on test set
preds = XOR_model.predict(X_test)
preds = np.round(preds[:, 0]).astype('float32')
acc = (np.sum(preds == y_test) / len(y_test)) * 100.
print('Accuracy: {:.2f}%'.format(acc))

# Plot model acc and loss
plot_model(history)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
_________________________________________________________________
dropout_2 (Dropout)          (None, 1)                 0         
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________
Train on 64000 samples, validate on 16000 samples
Epoch 1/25
64000/64000 [==============================] - 45s 701us/step - loss: 2.1786 - acc: 0.5001 - val_loss: 0.7141 - val_acc: 0.5022
Epoch 2/25
64000/64000 [==============================] - 41s 639us/step - loss: 2.1945 - acc: 0.4978 - val_loss: 0.7107 - val_acc: 0.5022
Epoch 3/25
64000/64000 [==============================] - 42s 660us/step - l